# Hands-On 1: Portable Parallel Programming with OpenMP

Team: João Vitor Mendes, Roberto Santana Santos

Welcome to Hands-on _Portable Parallel Programming with OpenMP_. This notebook comprises 2 sessions. Next table shows the documents and files needed to develop each one of the exercises.


|  Sessions     | Codes             | files           | 
| --------------| ----------------- | --------------- |
| Session 1     | Matrix Multiply   |  mm.c           |  
| Session 2     | Asynchronous Task |  asyncTaskOpenMP.c |   


## `Matrix Multiple Benchmark`

The definite algebraic operation of the matrix can be defined as:For your work today, you have access to several GPUs in the cloud. Run the following cell to see the GPUs available to you today.

$$ c_{ij} = \sum\limits_{k=1}^{n } a_{ik} b_{kj} = a_{i1}b_{1j} + a_{i2}b_{2j} + ... + a_{in}b_{nj} $$

where $i$ is summed over for all possible values of $j$ and $k$ and the notation above uses the summation convention. The sequential code of the program is available in the file `mm.c`. The follow code shows an extract of such code. In particular, we can see the algebraic operation include a loop that implements the summatory of the above definition.

In [1]:
%%writefile mm.c
#include <stdio.h>
#include <stdlib.h>

void initializeMatrix(int *matrix, int size)
{
  for(int i = 0; i < size; i++)
    for(int j = 0; j < size; j++)
      matrix[i * size + j] = rand() % (10 - 1) * 1;
}

void printMatrix(int *matrix, int size)
{
  for(int i = 0; i < size; i++)
  {
    for(int j = 0; j < size; j++)
      printf("%d\t", matrix[i * size + j]);
    printf("\n");
  }
  printf("\n");
}

int main(int argc, char **argv)
{
  int size = atoi(argv[1]);  
  int i, j, k;

  int  *A = (int *) malloc (sizeof(int)*size*size);
  int  *B = (int *) malloc (sizeof(int)*size*size);
  int  *C = (int *) malloc (sizeof(int)*size*size);

  initializeMatrix(A, size);
  initializeMatrix(B, size);

  for(i = 0; i < size; i++)
   for(j = 0; j < size; j++)
     for(k = 0; k < size; k++)
        C[i * size + j] += A[i * size + k] * B[k * size + j];

  printMatrix(A,size);
  printMatrix(B,size);
  printMatrix(C,size);

  return 0;
}

Overwriting mm.c


### Run the Code

In [2]:
!gcc mm.c -o mm

In [3]:
!./mm 10

1	7	0	7	5	7	1	3	6	1	
5	4	5	7	5	4	6	0	7	1	
8	8	6	6	8	8	8	4	1	1	
5	0	0	3	5	3	1	7	4	7	
6	0	0	2	5	4	5	2	2	3	
2	1	1	8	8	0	5	5	4	4	
6	0	5	6	2	8	7	3	4	2	
0	0	0	0	2	6	2	5	6	5	
7	6	6	8	5	3	6	2	8	1	
6	6	8	0	1	1	7	0	3	2	

0	1	2	1	8	3	5	2	6	0	
7	2	7	2	8	1	6	5	1	5	
4	6	0	4	6	2	3	2	0	4	
3	7	5	3	6	5	4	2	5	2	
1	3	2	8	3	2	0	0	7	2	
4	3	6	2	5	1	2	6	4	2	
2	7	8	5	1	5	1	4	8	4	
6	7	5	8	6	0	8	4	0	7	
4	2	8	1	5	2	3	7	8	7	
8	1	3	7	5	2	3	6	6	0	

155	141	212	132	210	81	135	157	173	140	
138	176	214	146	225	124	130	157	230	145	
190	237	261	226	306	142	192	189	252	175	
142	121	149	174	186	70	133	136	174	103	
81	103	131	124	137	77	82	100	171	70	
131	172	176	193	177	106	120	119	205	124	
136	188	203	153	215	117	134	162	212	129	
124	90	144	119	123	42	87	136	132	101	
171	208	249	174	281	140	176	184	253	180	
121	129	148	112	184	88	114	125	145	115	



### Entering time measurement metrics

The next step will be to modify the code in the file `mm.c` to enter time measurement metrics of the matrix multiply in parallel using OpenMP. A first approach could be to use the command `omp_get_wtime()` for the get the initial and final time. You will need to link the command with the OpenMP library by including `omp.h`.

In [28]:
%%writefile mm.c
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

void initializeMatrix(int *matrix, int size)
{
  for(int i = 0; i < size; i++)
    for(int j = 0; j < size; j++)
      matrix[i * size + j] = rand() % (10 - 1) * 1;
}

void printMatrix(int *matrix, int size)
{
  for(int i = 0; i < size; i++)
  {
    for(int j = 0; j < size; j++)
      printf("%d\t", matrix[i * size + j]);
    printf("\n");
  }
  printf("\n");
}

int main(int argc, char **argv)
{
 int size = atoi(argv[1]);  
 int i, j, k;
 double t1, t2;

 int  *A = (int *) malloc (sizeof(int)*size*size);
 int  *B = (int *) malloc (sizeof(int)*size*size);
 int  *C = (int *) malloc (sizeof(int)*size*size);

 initializeMatrix(A, size);
 initializeMatrix(B, size);

 t1 = omp_get_wtime();
   for(i = 0; i < size; i++)
    for(j = 0; j < size; j++)
      for(k = 0; k < size; k++)
        C[i * size + j] += A[i * size + k] * B[k * size + j];
 t2 = omp_get_wtime();

 printf("%d\t%f\n",size, t2-t1);

 //printMatrix(A,size);
 //printMatrix(B,size);
 //printMatrix(C,size);

 return 0;
}

Overwriting mm.c


### Run the Code

In [29]:
!gcc mm.c -o mm -fopenmp

In [40]:
!./mm 1000

1000	5.597884


### Inserting the OpenMP directive

The next step will be to modify the code in the file `mm.c` to perform the computation of the integral in parallel using OpenMP. A first approach could be to use the directive parallel for considering the variables $i$, $j$ and $k$ are private. After this change, you can compile and execute the program.

In [7]:
%%writefile mm.c
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

void initializeMatrix(int *matrix, int size)
{
  for(int i = 0; i < size; i++)
    for(int j = 0; j < size; j++)
      matrix[i * size + j] = rand() % (10 - 1) * 1;
}

void printMatrix(int *matrix, int size)
{
  for(int i = 0; i < size; i++)
  {
    for(int j = 0; j < size; j++)
      printf("%d\t", matrix[i * size + j]);
    printf("\n");
  }
  printf("\n");
}

int main (int argc, char **argv)
{
 int size = atoi(argv[1]);  
 int i, j, k;
 double t1, t2;

 int  *A = (int *) malloc (sizeof(int)*size*size);
 int  *B = (int *) malloc (sizeof(int)*size*size);
 int  *C = (int *) malloc (sizeof(int)*size*size);

 initializeMatrix(A, size);
 initializeMatrix(B, size);

 t1 = omp_get_wtime();
 #pragma omp parallel for private(i, j, k)
   for(i = 0; i < size; i++)
    for(j = 0; j < size; j++)
      for(k = 0; k < size; k++)
        C[i * size + j] += A[i * size + k] * B[k * size + j];
 t2 = omp_get_wtime();

 printf("%d\t%f\n",size, t2-t1);

 //printMatrix(A,size);
 //printMatrix(B,size);
 //printMatrix(C,size);

 return 0;
}

Overwriting mm.c


### Run the Code

In [8]:
!gcc mm.c -o mm -fopenmp

In [9]:
!OMP_NUM_THREADS=16 ./mm 1000

1000	1.548553


### Performance Analysis

The last step will be to create the shell script file to measure the perform the computation of the matrix multiply in parallel using OpenMP. The shell script is available in the file `script.sh`.
Compile and execute the script. At run time, an argument can be used to select the number of the threads. For example, to use the first variant you can use for $16$ threads:

In [10]:
%%writefile script.sh
#!/bin/sh

for i in 100 200 300 400 500 600 700 800 900 1000
do
  OMP_NUM_THREADS=$1  ./mm  "$i"
done

Writing script.sh


In [11]:
!bash script.sh 16

100	0.001791
200	0.007148
300	0.024069
400	0.059331
500	0.104449
600	0.171973
700	0.273477
800	0.465969
900	0.794182
1000	1.383371


Through experiments, the following questions should be answered:

* What is the behavior of the execution time and the speedup varying the size of the problem? (Show the solution with tabular and graphical values).

* What is the optimal number of threads for the best parallel solution?

### Behavior of the execution time and speedup for diferrent problem sizes

In order to analyze the behavior of the execution time and Speedup in the problem, it is necessary to understand that the execution time and Speedup will vary along with the increase in the size of the problem, some behaviors can already be expected, such as the use of a large number of threads in a small problem will cause a reduced Speedup, thus making code parallelization inefficient under these conditions. 

For our problem, we must test different execution sizes for an already defined set of threads. We will use a set of 16 threads, and an execution size ranging from 100 to 1000. From the execution of the experiments, we obtained the tables below.

|  Probelem Size     | Sequential Time    |  Paralel Time    | Speedup    |
|  -------------     | -------------     |  -------------     | -------------     |
|  100     | 0.003330     |  0.003987     | 0.83     |
|  200     | 0.028392     |  0.009018    | 3.14     |
|  300     | 0.112362    |  0.025856     | 4.34|
|  400     | 0.257749  |  0.053487    | 4.81|
|  500     | 0.537874   |  0.113625    | 4.73|
|  600     |0.972337    | 0.227770    | 4.26|
|  700     |1.676479    |  0.308470     | 5.43|
|  800     |2.484594    | 0.460911    | 5.51|
|  900     |3.751935    |  0.788040    | 4.76|
|  1000     |5.597884   | 1.447922    | 3.86|

### Finding the Optimal Number of Threads

We will find the optimal number of threads for the program through experimentation, by trying out different numbers of thread on our previously stated script, as shown in the table below.

Execution Size - 1000                              
|  Number of Threads     | Execution Time    |
| ------------------     | ----------------- |
| 2                      |   2.795087        |                             
|  4                     | 1.592035          |
| 6                      | 1.367070          |
| 8                      | 1.347624          |                                
|  10                    | 1.284874          |
| 12                     | 1.388694          |
| 14                     | 1.352184          |  
|16                      | 1.412781          |


According to the table, we can infer that the optimal number of threads required would be 10, although the ideal number depends on different hardware and software factors, so it cannot be considered a definitive or universal value.

In [14]:
!bash script.sh 12

100	0.004126
200	0.009398


300	0.027010
400	0.056464
500	0.111435
600	0.176179
700	0.286987
800	0.437534
900	0.805619
1000	1.388694


## `Asynchronous Task`

Asynchronous programming is a set of techniques for implementing expensive operations that run concurrently with the rest of the program. One domain where asynchronous programming is often used is in programs with a graphical user interface: it is often unacceptable when the user interface freezes while performing a costly operation. Also, asynchronous operations are essential for parallel applications that need to run multiple tasks simultaneously. The following is a code `asyncTaskOpenMP.c` that represents a task being done asynchronously. Before understanding the code, compile and run it as follows:

In [12]:
%%writefile asyncTaskOpenMP.c
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

#define SIZE_MATRIX 10

int main(int argc, char **argv)
{
  int n = atoi(argv[1]);
  int block_size = atoi(argv[2]);
  int matrix[SIZE_MATRIX][SIZE_MATRIX], k1 = 10, k2 = 20;
  int i, j, row, column;

  for(i = 0; i < n; i++)
  {
    for(j = 0; j < n; j++)
    {
      matrix[i][j] = 5;
      printf("%d\t", matrix[i][j]);
    }
    printf("\n");
  }

  printf("\n\n");

  omp_set_num_threads(5);

  #pragma omp parallel private(row, column)
  {
    int id = omp_get_thread_num();

    if(id == 0)
    {
      for(row = 0; row < n; row++)
        for(column = 0; column < block_size; column++)
          matrix[row][column] *= k1;
    }

    if(id == 1)
    {
      for(row = 0; row < n; row++)
        for(column = block_size; column < 2 * block_size; column++)
          matrix[row][column] *= k2;
    }
  
  }

  for(i = 0; i < n; i++)
  {
    for(j = 0; j < n; j++)
      printf("%d\t", matrix[i][j]);
    printf("\n");
  }

  return 0;
}

Writing asyncTaskOpenMP.c


### Run the Code

In [13]:
!gcc asyncTaskOpenMP.c -o asyncTaskOpenMP -fopenmp

In [14]:
!./asyncTaskOpenMP 10 2

5	5	5	5	5	5	5	5	5	5	
5	5	5	5	5	5	5	5	5	5	
5	5	5	5	5	5	5	5	5	5	
5	5	5	5	5	5	5	5	5	5	
5	5	5	5	5	5	5	5	5	5	
5	5	5	5	5	5	5	5	5	5	
5	5	5	5	5	5	5	5	5	5	
5	5	5	5	5	5	5	5	5	5	
5	5	5	5	5	5	5	5	5	5	
5	5	5	5	5	5	5	5	5	5	


50	50	100	100	5	5	5	5	5	5	
50	50	100	100	5	5	5	5	5	5	
50	50	100	100	5	5	5	5	5	5	
50	50	100	100	5	5	5	5	5	5	
50	50	100	100	5	5	5	5	5	5	
50	50	100	100	5	5	5	5	5	5	
50	50	100	100	5	5	5	5	5	5	
50	50	100	100	5	5	5	5	5	5	
50	50	100	100	5	5	5	5	5	5	
50	50	100	100	5	5	5	5	5	5	


From the previous action answer the following questions:

• What does the code do from the compilation and execution of the previous code?

• How would it be possible to extend the code so that the five threads perform asynchronous tasks?

### Code Explanation

The code presented above seeks to divide the matrix into parts, with a size determined by the block_size parameter, in which each thread will be responsible for performing operations on these blocks independently.

### Extending the Code for the Five Threads to Perform Asynchronous Tasks

To extend the code for all threads perform asynchronously, we only need to adjust the range each will follow and adjust the if-statements to cover all IDs, as shown below.

In [15]:
%%writefile asyncTaskOpenMP.c
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

#define SIZE_MATRIX 10

int main(int argc, char **argv)
{
  int n = atoi(argv[1]);
  int block_size = atoi(argv[2]);
  int matrix[SIZE_MATRIX][SIZE_MATRIX], k1 = 10, k2 = 20, k3 = 30, k4 = 40, k5 = 50;
  int i, j, row, column;

  for(i = 0; i < n; i++)
  {
    for(j = 0; j < n; j++)
    {
      matrix[i][j] = 5;
      printf("%d\t", matrix[i][j]);
    }
    printf("\n");
  }

  printf("\n\n");

  omp_set_num_threads(5);

  #pragma omp parallel private(row, column)
  {
    int id = omp_get_thread_num();

    if(id == 0)
    {
      for(row = 0; row < n; row++)
        for(column = 0; column < block_size; column++)
          matrix[row][column] *= k1;
    }

    if(id == 1)
    {
      for(row = 0; row < n; row++)
        for(column = block_size; column < 2 * block_size; column++)
          matrix[row][column] *= k2;
    }

    if(id == 2)
    {
      for(row = 0; row < n; row++)
        for(column = 2 * block_size; column < 3 * block_size; column++)
          matrix[row][column] *= k3;
    }

    if(id == 3)
    {
      for(row = 0; row < n; row++)
        for(column = 3 * block_size; column < 4 * block_size; column++)
          matrix[row][column] *= k4;
    }

    if(id == 4)
    {
      for(row = 0; row < n; row++)
        for(column = 4 * block_size; column < 5 * block_size; column++)
          matrix[row][column] *= k5;
    }
  
  }

  for(i = 0; i < n; i++)
  {
    for(j = 0; j < n; j++)
      printf("%d\t", matrix[i][j]);
    printf("\n");
  }

  return 0;
}

Overwriting asyncTaskOpenMP.c


In [16]:
!gcc asyncTaskOpenMP.c -o asyncTaskOpenMP -fopenmp

In [17]:
!./asyncTaskOpenMP 10 2

5	5	5	5	5	5	5	5	5	5	
5	5	5	5	5	5	5	5	5	5	
5	5	5	5	5	5	5	5	5	5	
5	5	5	5	5	5	5	5	5	5	
5	5	5	5	5	5	5	5	5	5	
5	5	5	5	5	5	5	5	5	5	
5	5	5	5	5	5	5	5	5	5	
5	5	5	5	5	5	5	5	5	5	
5	5	5	5	5	5	5	5	5	5	
5	5	5	5	5	5	5	5	5	5	


50	50	100	100	150	150	200	200	250	250	
50	50	100	100	150	150	200	200	250	250	
50	50	100	100	150	150	200	200	250	250	
50	50	100	100	150	150	200	200	250	250	
50	50	100	100	150	150	200	200	250	250	
50	50	100	100	150	150	200	200	250	250	
50	50	100	100	150	150	200	200	250	250	
50	50	100	100	150	150	200	200	250	250	
50	50	100	100	150	150	200	200	250	250	
50	50	100	100	150	150	200	200	250	250	


## References

M. Boratto. Hands-On Supercomputing with Parallel Computing. Available: https://github.com/muriloboratto/Hands-On-Supercomputing-with-Parallel-Computing. 2022.

B. Chapman, G. Jost and R. Pas. Using OpenMP: Portable Shared Memory Parallel Programming. The MIT Press, 2007, USA.